<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale - Generate Configuration Archive

This notebook demonstrates how to generate a configuration archive for monitoring deployments in IBM Watson OpenScale. This configuration is targetted for `System-Managed` monitored deployments.

***Target audience for this notebook:***
This notebook is targetted for users who fall in either of the below categories:
- Users who cannot provide training data (*as CSV or in DB2 or COS*) while configuring a deployment for monitoring in IBM Watson OpenScale
- Users who have large amount of training data (> 500MB) and as such can not be used for creating artifacts in IBM Watson OpenScale
- Users who are looking for automation and/or more granular control using Python SDK.

User must provide the necessary inputs where marked. Generated configuration package can be used in IBM Watson OpenScale UI while configuring monitoring of a model deployment in IBM Watson OpenScale.

**Contents:**
1. [Setting up the environment](#setting-up-the-environment) - Pre-requisites: Install Libraries and required dependencies
2. [Training Data](#training-data) - Read the training data as a pandas DataFrame
3. [User Inputs Section](#user-inputs-section) - Provide Model Details, IBM Watson OpenScale Services and their configuration
4. [Generate Configuration Archive](#generate-configuration-archive)
5. [Helper Methods](#helper-methods)
6. [Definitions](#definitions)

## Setting up the environment

In [ ]:
# When this notebook is to be run on a zLinux cluster,
# install scikit-learn==1.1.1 using conda before installing ibm-metrics-plugin
# !conda install scikit-learn=1.1.1
# !conda install -y cmake cython==0.29.33 openblas==0.3.21 qdldl-python==0.1.7 pandas==1.4.4 pyparsing==2.4.7 statsmodels==0.13.2
# !git clone https://github.com/tommyod/KDEpy && cd KDEpy && git checkout d52233099978dec38fa622fc86ce5e10368db1bd && rm -f KDEpy/cutils.c && CC=gcc python -m pip install . && cd .. && rm -rf KDEpy
# !CC=gcc python -m pip install jenkspy==0.2.0 retrying==1.3.4 marshmallow==3.10.0 more-itertools==8.12.0 numba==0.57.1
# !python -m pip install --no-build-isolation osqp==0.6.2 cvxpy==1.3.2
# !python -m pip install shap==0.41.0
# !CC=gcc GXX=g++ python -m pip install ibm-metrics-plugin~=5.0.3

# If running the notebooks against Default Spark runtimes on CP4D clusters, please replace the following command
%pip install --upgrade "ibm-metrics-plugin~=5.1.0" "ibm-watson-openscale>=3.0.31"  

# with
# %pip install --upgrade "ibm-metrics-plugin~=5.0.3.11" "ibm-watson-openscale>=3.0.32" -t /home/spark/shared/user-libs/python | tail -n 1


In [12]:
# ----------------------------------------------------------------------------------------------------
# IBM Confidential
# OCO Source Materials
# 5900-A3Q, 5737-H76
# Copyright IBM Corp. 2018, 2023
# The source code for this Notebook is not published or other-wise divested of its trade 
# secrets, irrespective of what has been deposited with the U.S.Copyright Office.
# ----------------------------------------------------------------------------------------------------
VERSION = "7.2.0"
# Version history
# 7.2.0 : Upgraded version of ibm-metrics-plugins to use  scikit version 1.5.1
# 7.1.0 : Update pip install to ~=4.8.0 and add input_data_type in common parameters
# 7.0.0 : Official notebook for IBM CP4D 4.8.x
#         Upgrade ibm-metrics-plugin to >= 4.8.0;
# 6.0.0 : Complete Re-design of the notebook; Official notebook for IBM CP4D 4.7.x
#         Upgrade ibm-metrics-plugin to >= 4.7.0; Includes Support for Drift v2 Archive
#         Refactored for Configuration Package; Deprecate the Drift Archive
# 5.4.7 : Official notebook for IBM CP4D 4.6.4
#         Upgrade ibm-metrics-plugin to >= 4.6.4.0
# 5.4.6 : Take optional class labels input for global explainability
# 5.4.5 : Remove numpy and scipy versions to be installed.
# 5.4.4 : Add support for lime global explanation
# 5.4.3 : Add numpy and scipy versions to be installed.
# 5.4.2 : Remove explainability configuration while saving training_distribution
# 5.4.1 : Add sample size for generating global explanation
# 5.4.0 : Add support for SHAP Global explanation
# 5.3.6 : Fix issue with explainability archive generation for regression model
# 5.3.5 : Official notebook for IBM CPD 4.5.0. 
#         Upgrade ibm-wos-utils to 4.5.0. 
#         Added code to generate explainability perturbations archive.
# 5.3.4 : Upgrade ibm-wos-utils to 4.1.1 (scikit-learn has been upgraded to 1.0.2)
# 5.3.3 : Upgrade ibm-wos-utils to 4.0.34
# 5.3.2 : Upgrade ibm-wos-utils to 4.0.31
# 5.3.1 : Official notebook for IBM CPD 4.0.5

## Training Data
*Note: Pandas' read\_csv method converts the columns to its data types. If you want the column type to not be interpreted, specify the dtype param to read_csv method in this cell. More on this method [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)*

*Note: By default NA values will be dropped while computing training data distribution. Please ensure to handle the NA values during Pandas' read\_csv method*

In [14]:
import pandas as pd
training_data_df = pd.read_csv("<EDIT THIS>")

print(training_data_df.head())
print("Columns:{}".format(list(training_data_df.columns.values)))

  CheckingStatus  LoanDuration           CreditHistory LoanPurpose  \
0         less_0            21  prior_payments_delayed   furniture   
1    no_checking            18  prior_payments_delayed    car_used   
2    no_checking             4   all_credits_paid_back       other   
3         less_0            12  prior_payments_delayed    car_used   
4         less_0            21  prior_payments_delayed    radio_tv   

   LoanAmount ExistingSavings EmploymentDuration  InstallmentPercent     Sex  \
0        9235        less_100             4_to_7                   4  female   
1        7165        less_100             1_to_4                   2    male   
2        6095        less_100             4_to_7                   4    male   
3        5025     500_to_1000             4_to_7                   1    male   
4        7326        less_100             4_to_7                   3  female   

   OthersOnLoan  ...  InstallmentPlans Housing  ExistingCreditsCount  \
0  co-applicant  ...      

## User Inputs Section

##### _1. Provide Common Parameters_:

Provide the common parameters like the basic model details like type, feature columns, etc. Also, enable/disable the different monitors you would like th artifacts for. Read more about these [here](#common-parameters). 

##### _2. Provide Fairness Parameters_
The fairness parameters are required if `enable_fairness` is set to `True`. Read more about these parameters [here](#fairness-parameters)

##### _3. Provide Explainability Parameters_
The explainability parameters are required if `enable_explainability` is set to `True`. Read more about these parameters [here](#explainability-parameters)

*When LIME global explanation is enabled, the explainability archive upload and explainability monitor enablement should be done using python sdk/api.*

##### _4. Provide Drift v2 Parameters_
Read more about these parameters [here](#drift-v2-parameters)

##### _5. *DEPRECATED* Provide Drift Parameters_
Read more about these parameters [here](#deprecated-drift-parameters)

##### _6. Provide a scoring function_
The scoring function is required if any of  `enable_explainability`, `enable_drift_v2` or `enable_drift` is set to `True`. The scoring function should adhere to the following guidelines.

- The input of the scoring function should accept a `pandas.DataFrame` containing all the `feature_columns` used to build the model.
- The output of the scoring function should return:
    - a `tuple` of `(probabilities, predictions)` for classification problems. Both `probabilities` and `predictions` are of type `numpy.ndarray`
    - a `numpy.ndarray` of `predictions` for regression problems.
- The data type of the label column and prediction column should be same. Moreover, the label column and the prediction column array should have the same unique class labels
- A host of different scoring function templates are provided [here](https://github.com/IBM/watson-openscale-samples/wiki/Score-function-templates-for-IBM-Watson-OpenScale)

In [33]:
common_parameters = {
    "problem_type" : "TO_BE_EDITED",
    "input_data_type": "structured",
    "label_column": "TO_BE_EDITED",
    "prediction_column": "TO_BE_EDITED",
    "probability_column": "TO_BE_EDITED",     # <- Not required for Regression problems.
    "class_probabilities": ["TO_BE_EDITED"],  # <- Optional. Not required for Regression problems.
    "feature_columns": ["TO_BE_EDITED"],      # <- If not provided, all columns in data frame except 
                                              # (label, prediction and probability) columns, will be
                                              # marked as feature_columns
    "categorical_columns": ["TO_BE_EDITED"],  # <- If not provided, all columns of dtype object 
                                              # and bool in feature columns, will be marked as
                                              # categorical_columns.
    "enable_quality": True,
    "enable_fairness": True,
    "enable_explainability": True,
    "enable_drift_v2": True,
    # "enable_drift": False, # <- set this to True for creating the archive for the DEPRECATED Drift monitor
    "notebook_version": VERSION
}

fairness_parameters = {
    "fairness_attributes": [
        {
            "type": "TO_BE_EDITED",
            "feature": "TO_BE_EDITED",
            "majority": [
                "TO_BE_EDITED"
            ],
            "minority": [
                "TO_BE_EDITED"
            ],
            "threshold": 0.8
        }
    ],
    "min_records" : "TO_BE_EDITED",
    "max_records": None,
    "favourable_class" : ["TO_BE_EDITED"],
    "unfavourable_class": ["TO_BE_EDITED"]
}

explainability_parameters = {
    "global_explanation": {
        "enabled": True,
        "sample_size": 50,
        "training_data_sample_size": 100,
        "explanation_method": "lime"
    },
    "shap": {
        "enabled": False,
#         "perturbations_count": 100,
#         "background_data_set": None,
#         "background_data_sets": []
    },
    "lime": {
        "enabled": True,
#         "perturbations_count": 5000
    },
    "local_explanation_method": "lime",
}


drift_v2_parameters = {
    # "max_samples": 10000
}

# Drift is being deprecated in favour of Drift v2.
# drift_parameters = {
#     "data_drift": {
#         "two_column_learner_limit": 200,
#         "categorical_unique_threshold": 0.8,
# #         "user_overrides": [
# #             {
# #                 "constraint_type": "TO_BE_EDITED",
# #                 "learn_distribution_constraint": "TO_BE_EDITED",
# #                 "learn_range_constraint": "TO_BE_EDITED",
# #                 "features": ["TO_BE_EDITED"]
# #             }
# #         ]
#     },
#     "model_drift": {
#         "optimise": True,
#         "check_for_ddm_quality": False,
#         "ddm_quality_check_threshold": 0.3
#     }
# }

scoring_fn = None

## Generate Configuration Archive

Run the following code to generate the configuration archive for the IBM Watson OpenScale monitors. This archive is used as is by IBM Watson OpenScale UI/SDK to onboard model for monitoring. UI/SDK will identify the different artifacts and appropriately upload to respective monitors.

In [36]:
from ibm_watson_openscale.utils.configuration_utility import ConfigurationUtility

config_util = ConfigurationUtility(
    training_data=training_data_df,
    common_parameters=common_parameters,
    scoring_fn=scoring_fn if "scoring_fn" in locals() else None)

config_util.create_configuration_package(
    explainability_parameters=explainability_parameters if "explainability_parameters" in locals() else None,
    drift_v2_parameters=drift_v2_parameters if "drift_v2_parameters" in locals() else {},
    fairness_parameters=fairness_parameters if "fairness_parameters" in locals() else None,
    display_link=True )

Training Statistics generated.


Drift v2 Archive generated.


## Helper Methods

### Read file in COS to pandas dataframe

In [ ]:
%pip install ibm-cos-sdk

import ibm_boto3
import pandas as pd
import sys
import types

from ibm_botocore.client import Config

def __iter__(self): return 0

api_key = "TO_BE_EDITED" # cos api key
resource_instance_id = "TO_BE_EDITED" # cos resource instance id
service_endpoint =  "TO_BE_EDITED" # cos service region endpoint
bucket =  "TO_BE_EDITED" # cos bucket name
file_name= "TO_BE_EDITED" # cos file name
auth_endpoint = "https://iam.ng.bluemix.net/oidc/token"

cos_client = ibm_boto3.client(service_name="s3",
    ibm_api_key_id=api_key,
    ibm_auth_endpoint=auth_endpoint,
    config=Config(signature_version="oauth"),
    endpoint_url=service_endpoint)

body = cos_client.get_object(Bucket=bucket,Key=file_name)["Body"]

# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

training_data_df = pd.read_csv(body)

## Definitions

### Common Parameters

| Parameter | Description | Default Value | Possible Value(s) |
|:-|:-|:-|:-|
| model_type | Enumeration classifying if your model is a binary or a multi-class classifier or a regressor. |  | `binary`, `multiclass`, `regression` |
| label_column | The column which contains the target field (also known as label column or the class label). |  | A string value referring column name |
| feature_columns | Columns identified as features by model. The order of the feature columns should be same as that of the subscription. Use helper methods to compute these if required. |  | A list of column names |
| categorical_columns | Feature columns identified as categorical by model. Use helper methods to compute these if required. |  | A list of column names |
| prediction_column | The column containing the model output. This should be of the same data type as the label column. |  | A string value referring column name |
| probability_column | The column (of type array) containing the model probabilities for all the possible prediction outcomes. This is not required for regression models. One of `probability_column` or `class_probabilities` must be specified for classification models. If both are specified, `class_probabilities` is preferred.|  | A string value referring column name |
| class_probabilities | The columns (of type double) containing the model probabilities of class labels. This is not required for regression models. For example, for Go Sales model deployed in MS Azure ML Studio, value of this property would be `["Scored Probabilities for Class \"Camping Equipment\"", "Scored Probabilities for Class \"Mountaineering Equipment\"", "Scored Probabilities for Class \"Personal Accessories\""]`. Please note escaping double quotes is a must-have requirement for above example. One of `probability_column` or `class_probabilities` must be specified for classification models. If both are specified, `class_probabilities` is preferred. |  | A list of column names |
| enable_quality | Boolean value to enable the Quality monitor | `True` | `True` or `False` |
| enable_fairness | Boolean value to allow generation of fairness specific data distribution needed for configuration | `True` | `True` or `False` |
| enable_explainability | Boolean value to allow generation of explainability configuration | `True` | `True` or `False` |
| enable_drift_v2 | Boolean value to allow generation of Drift v2 Archive. | `True` | `True` or `False` |
| enable_drift | Boolean value to allow generation of *DEPRECATED* Drift Archive containing relevant information for Model and Data Drift. | `False` | `True` or `False` |


### Fairness Parameters
Provide the fairness parameters. Leave the variable `fairness_parameters` to `None` or `{}` if fairness is not to be enabled.

| Parameter | Description | Default Value | Possible Value(s) |
| :- | :- | :- | :- |
| fairness_attributes.type | Data type of the fairness attribute. | | `float`, `int`, `double`, `string`, etc.|
| fairness_attributes.feature | Defines feature to monitor for fairness. | | |
| fairness_attributes.majority | Defines majority group towards which the model might be biased. | | |
| fairness_attributes.minority | Defines minority group for which we want to ensure that the model is not biased. | | |
| fairness_attributes.threshold | Defines value beyond which the model is considered to be biased. | | |
| min_records | Number of (latest) records to use for computing fairness. If we set the value of "min_records" to a small number, then fairness computation will get influenced by the scoring requests sent to the model in the recent past. In other words, the model might be flagged as being biased if it is acting in a biased manner on the last few records, but overall it might not be acting in a biased manner. On the other hand, if the "min_records" is set to a very large number, then we will not be able to catch model bias quickly. Hence the value of min_records should be set such that it is neither too small or too large. | | |
| max_records | Optional parameter. | | |
| favourable_class | Class labels considered to be expected outcome. For regression models, this is defined as range of values. | | |
| unfavourable_class | Class labels considered to be un-expected outcome. For regression models, this is defined as range of values. | | |

Example:
For a Loan Processing Model:
- We want to ensure that the model is not biased against people of specific age group and people belonging to a specific gender. Hence `Applicant_Age` and `Gender` will be the fairness attributes for this model.
- Majority group for the fairness attribute `Applicant_Age` is defined as `[31,60]`, i.e., all the ages except the minority group. For the fairness attribute `Gender`, the majority group is defined as `Male`. 
- Further, we want to ensure that model is not biased against people in the age group 15 to 30 years & 61 to 120 years as well as people with Gender = Female or Gender = Transgender. Hence, minority group for the fairness attribute `Applicant_Age` is defined as `[15,30]` and `[61,120]` and minority group for fairness attribute `Gender` is defined as `Female`, `Transgender`.
- Let us say that the Bank is willing to tolerate the fact that Female and Transgender applicants will get up to 20% lesser approved loans than Males. However, if the percentage is more than 20% then the Loan Processing Model will be considered biased. E.g., if the percentage of approved loans for Female or Transgender applicants is say 25% lesser than those approved for Male applicants then the Model is to be considered as acting in a biased manner. Thus for this scenario, the Fairness threshold will be `80` (100-20) (this is represented as a value normalized to 1, i.e., 0.8).
- In case of Loan Processing Model, the target field (label column or class label) can have the following values: `Loan Granted`, `Loan Denied` and `Loan Partially Granted`. Out of these values `Loan Granted` and `Loan Partially Granted` can be considered as being favorable and `Loan Denied` is unfavorable. In other words in order to measure fairness, we need to know the target field values which can be considered as being favourable and those values which can be considered as unfavourable.
- In case of a regression models, the favourable and unfavourable classes will be ranges. For example, for a model which predicts medicine dosage, the favorable outcome could be between 80 ml to 120 ml or between 5 ml to 20 ml whereas unfavorable outcome will be values between 21 ml to 79ml.
- Fairness checks runs hourly. If `min_records` is set to `5000`, then every hour fairness checking will pick up the last 5000 records which were sent to the model for scoring and compute fairness using these. Please note that fairness computation will not start till the time that 5000 records are sent to the model for scoring.

```json
{
    "fairness_attributes": [
        {
            "feature": "Applicant_Age",
            "type": "int",
            "majority": [
                [31, 60]
            ],
            "minority": [
                [15, 30],
                [61, 120]
            ],
            "threshold": 0.8
        },
        {
            "feature": "Gender",
            "type": "string",
            "majority": ["Male"],
            "minority": ["Female", "Transgender"],
            "threshold": 0.8
        }
    ],
    "min_records": 5000,
    "max_records": null,
    "favourable_class": ["Loan Granted", "Loan Partially Granted"],
    "unfavourable_class": ["Loan Denied"]
}
```

- For regression problems, set `favourable_class` and `unfavourable_class` as shown below:

```json
{
    "favourable_class" : [[5, 20], [80, 120]],
    "unfavourable_class": [[21, 79]]
}
```


### Explainability Parameters
Provide the explainability parameters. Leave the variable `explainability_parameters` to `None` or `{}` if explainability is not to be enabled.

**Note: LIME global explanation feature is supported from Cloud Pak for Data version 4.6.4 onwards**

*When LIME global explanation is enabled, the explainability archive upload and explainability monitor enablement should be done using python sdk/api.*

| Parameter | Description | Default Value | Possible Value(s) |
| :- | :- | :- | :- |
| global_explanation | The global explanation parameters. | | |
| global_explanation.enabled | Boolean value to enable of disable global explanation. | | `True` or `False` |
| global_explanation.sample_size | The sample size of the records to be considered for computing global explanation in the payload window. | | |
| global_explanation.training_data_sample_size | The sample size of the records to be considered for computing global explanation on the training data. | 1000| |
| global_explanation.explanation_method | Type of technique to use for generating global explanation. SHAP parameters should be provided when shap explanation method is selected. | `lime` | `shap` or `lime` |
| shap | The shap explanation parameters are **mandatory** when SHAP explanation method is selected for local or global explanation. | | |
| shap.enabled | Boolean value to enable or disable SHAP based explanations. | | `True` or `False` |
| shap.perturbations_count | Number of perturbations to generate during explanation generation. | 100 | |
| shap.background_data_set | The background data set to be used when generating the SHAP explanation of a transaction. The background data is used to determine the average predicted value for regression models and the average confidence value for classification models. When generating a local explanation, SHAP computes the shapley values which signify how much each feature contributed to moving the model output or model confidence from the computed average value. If not defined, it is auto-generated using training data. | | |
| shap.background_data_sets | A list of available background_data_set which can be used for configuration. | | |
| lime.enabled | Boolean value to enable or disable LIME based explanations. | | `True` or `False` |
| lime.perturbations_count | Number of perturbations to generate during explanation generation. | 5000 | |
| local_explanation_method | Set explanation method to use for generating local explanations. | | `shap` or `lime` |

Example:

```json
{
    "global_explanation": {
        "enabled": true,
        "sample_size": 50,
        "training_data_sample_size": 1000,
        "explanation_method": "shap"
    },
    "shap": {
        "enabled": true,
        "perturbations_count": 100,
        "background_data_set": "data_set_1",
        "background_data_sets": [
            {
                "name": "data_set_1",
                "file_name": "data_set_1.csv"
            }
        ]
    },
    "lime": {
        "enabled": true,
        "perturbations_count": 5000
    },
    "local_explanation_method": "shap"
}
```

### Drift v2 Parameters

| Parameter | Description | Default Value | Possible Value(s) |
| :- | :- | :- | :- |
| max_samples | Defines maximum sample size on which the drift v2 archive is created. | None | |

### *DEPRECATED* Drift Parameters

| Parameter | Description | Default Value | Possible Value(s) |
| :- | :- | :- | :- |
| two_column_learner_limit | Defines upper limit on number of two-column constraints to learn. | 200 | |
| categorical_unique_threshold | Defines the threshold on percentage of unique categories. Ignore categorical column from constraint learning process if breached. | 0.8 | |
| user_overrides | Define custom rules for including or excluding columns during constraining learning. | | |
| constraint_type | Identifies type of constraint to apply rule to. | | `single` or `double` |
| learn_distribution_constraint | Boolean value enabling or disabling distribution constraint learning. | | `True` or `False` |
| learn_range_constraint | Boolean value enabling or disabling range constraint learning. | | `True` or `False` |
| features | Define features or a combination of features to apply rule on. | | |

In the example below:
- first config block says do not learn distribution and range single column constraints for features `MARITAL_STATUS`, `PROFESSION`, `IS_TENT` and `age`.
- Second config block says do not learn distribution and range two column constraints where `IS_TENT`, `PROFESSION`, and `AGE` are one of the two columns. Whereas, specifically, do not learn two column distribution and range constraint on combination of `MARITAL_STATUS` and `PURCHASE_AMOUNT`.

```json
"user_overrides"= [
    {
        "constraint_type": "single",
        "learn_distribution_constraint": False,
        "learn_range_constraint": False,
        "features": [
          "MARITAL_STATUS",
          "PROFESSION",
          "IS_TENT",
          "age"
        ]
    },
    {
        "constraint_type": "double",
        "learn_distribution_constraint": False,
        "learn_range_constraint": False,
        "features": [
          [
            "IS_TENT"
          ],
          [
            "MARITAL_STATUS"
            "PURCHASE_AMOUNT"
          ],
          [
            "PROFESSION"
          ],
          [
            "AGE"
          ]
        ]
    }
]
```